In [365]:
# Import packages
import requests
from bs4 import BeautifulSoup
import math
import re
import time
import codecs


headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

In [269]:
# Get the HTML code
url = "https://tribunalsdecisions.service.gov.uk/utiac?&page=1"
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   Tribunal decisions
  </title>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <link href="/assets/favicon-5aa00a9f9e6dc48d4e92f306a94e9496.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
  <!-- For third-generation iPad with high-resolution Retina display: -->
  <link href="/assets/apple-touch-icon-144x144.png" rel="apple-touch-icon-precomposed" sizes="144x144"/>
  <!-- For iPhone with high-resolution Retina display: -->
  <link href="/assets/apple-touch-icon-114x114.png" rel="apple-touch-icon-precomposed" sizes="114x114"/>
  <!-- For first- and second-generation iPad: -->
  <link href="/assets/apple-touch-icon-72x72.png" rel="apple-touch-icon-precomposed" sizes="72x72"/>
  <!-- For non-Retina iPhone, iPod Touch, and Android 2.1+ devices: -->
  <link href="/assets/apple-touch-icon-57x57.png" rel="apple-touch-icon-precomposed"/>
  <script>
   (function(){if(navigator.userAgent.match(/IEMobile\/10\.0/)){va

In [138]:
# Retrieve relevant variables
total_cases = int(str(re.findall(r'<b>(.*)</b>', str(soup.find_all("b")[1]), re.DOTALL))[2:7])
div_pages = int(str(re.findall(r'-(.*)</b>', str(soup.find_all("b")[0]), re.DOTALL))[6:8])
total_pages = math.ceil(total_cases/div_pages)

In [494]:
print(total_cases)


36436


In [495]:
print(div_pages)

30


In [496]:
print(total_pages)

1215


In [361]:
def convert_list_to_dic(lst):
    res_dct = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)}
    return res_dct

In [493]:
# Retrieve **ALL** documents
id_case_list = []
url_case_pdf_list = []
details_cases = []

with open('../data/corpus_archive_asylum.txt', 'w') as f:
    for i in range(0,total_pages):
        print(i)
        url = 'https://tribunalsdecisions.service.gov.uk/utiac?&page=' + str(i+1)
        req = requests.get(url, headers)
        soup = BeautifulSoup(req.content, 'html.parser')  
        table_cases = soup.find_all("table", class_='decisions-table')
        list_cases = re.findall(r'<a href="(.*)"></a>', str(table_cases))
        for j in range(0, len(list_cases)):
            id_case_list.append(list_cases[j])
            url_case = 'https://tribunalsdecisions.service.gov.uk/' + list_cases[j]
            req_case = requests.get(url_case, headers)
            soup_case = BeautifulSoup(req_case.content, 'html.parser')  
            url_case_pdf_aux = str(soup_case.find_all("a", class_='pdf-file'))
            url_case_pdf = re.findall(r'href="(.*)">Download', url_case_pdf_aux)
            url_case_pdf_list.append(url_case_pdf)

            # Get the metadata
            meta_data_case = [item.get_text() for item in soup_case.select("span")]
            # Clean text:
            meta_data_case = [e.replace('\xa0', '') for e in meta_data_case]
            meta_data_case = [e.replace('\n', '') for e in meta_data_case]
            # Append metadata into list of cases
            meta_data_case = convert_list_to_dic(meta_data_case)
            details_cases.append(meta_data_case)

            # Save content on a .txt
            try: 
                text_case_aux = soup_case.find("div", class_='decision-inner').get_text(strip=True)
                f.write(text_case_aux)
                f.write('\n')
                f.write('\n')
                f.write('\n')
            except:
                url_case_doc_aux = str(soup_case.find_all("a", class_='doc-file'))
                url_case_doc = re.findall(r'href="(.*)">Download', url_case_doc_aux)
                urllib.request.urlretrieve(url_case_doc[0], 'asylum_case_doc')
                #print("Error!")
                #print(url_case)
                up_boolean = False
                crown_boolean = True
                with codecs.open('asylum_case_doc', 'r', encoding="ISO-8859-1") as f2:
                     for num, line in enumerate(f2, 1):
                        if('Upper Tribunal' in line):
                            up_boolean = True
                        if('CROWN COPYRIGHT' in line):
                            crown_boolean = False
                            f.write(line)
                        if up_boolean and crown_boolean:
                            f.write(line)
        time.sleep(10)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

HTTPError: HTTP Error 403: Forbidden